# Book: Machine Learning Using Python

**Authors: MANARANJAN PRADHAN & U DINESH KUMAR**

## Solutions for Exercises Chapter 9: Recommender Systems
**Prepared By: Midhun Angathil**  

### Answer Questions 1 to 5 using the Dataset *Online Retail.xlsx.* 

The dataset Online Retail.xlsx and the description of the data is taken from https://archive.ics.uci.edu/ml/datasets/online+retail

Online Retail.xlsx contains records of transactions that occured between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholsalers. The attributes in the dataset are:

* *InvoiceNo -* Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts     with letter 'c', it indicates a cancellation.
* *StockCode -* Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
* *Description -* Product (item) name. Nominal.
* *Quantity -* The quantities of each product (item) per transaction. Numeric.
* *InvoiceDate -* Invoice Date and time. Numeric, the day and time when each transaction was generated.
* *UnitPrice -* Unit price. Numeric, Product price per unit in sterling.
* *CustomerID -* Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
* *Country -* Country name. Nominal, the name of the country where each customer resides.

### Note: 

The code cells that follow in this notebook require high computational power. Hence, use of free tools such as Google Colab (with Hardware accelerator: TPU) is recommended to avoid potential system hangs. 

In [5]:
#Mount the google drive after uploading datasets
from google.colab import drive
drive.mount('/content/gdrive')  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
#switch to the directory in the google drive to import datasets
cd /content/gdrive/My Drive/

/content/gdrive/My Drive


### Data Exploration and Preparation

In [0]:
#import necessary libraries
import pandas as pd
import numpy as np

#Reading data
data = pd.read_excel('/content/gdrive/My Drive/My solution for Chapter 9 (Machine Learning Using Python)/Online Retail.xlsx')

In [0]:
#Exploring the top 5 rows of the dataset
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [0]:
#Checking the shape of the dataset
data.shape

(541909, 8)

We have a total of 541909 rows and 8 columns in this dataset

We now check for missing values in the dataset

In [0]:
#Checking for missing values (if any)
data.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Description column can be ignored. However, CustomerID cannot be null. Hence, retaining only those customers that have a CustomerID mentioned.

In [0]:
#Retaining non-null CustomerIDs and ignoring Description
data = data[pd.notnull(data['CustomerID'])]

Some more clean ups that need to be done: First, some of the descriptions have spaces that need to be removed. We’ll also drop the rows that don’t have invoice numbers and remove the credit transactions (those with invoice numbers containing C).

In [0]:
#Removing spaces of some of those descriptions 
data['Description'] = data['Description'].str.strip()

#Removing rows that do not have invoice numbers 
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

#Remove credit transactions (those with invoice numbers containing C)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')]

Quantity cannot be negative. Hence, checking for the same:



In [0]:
#Checking for negative values in Quantity
data.Quantity.min()

1

Similarly Unit Price cannot be negative. Checking for the same:

In [0]:
#Checking if Unit Price is negative
data.UnitPrice.min()

0.0

In [0]:
#Converting type of InvoiceDate to datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [0]:
#Checking shape of data
data.shape

(397924, 8)

Turning our attention to StockCode, We notice a handful of non-product related codes (‘Postage’, ‘Bank Charges’, ‘Gift Vouchers’, etc.).

In [0]:
#Converting StockCode type
data['StockCode'] = data['StockCode'].astype('str')

#Excluding rows that match the specified non-product related codes
data = data[~data['StockCode'].str.contains('AMAZONFEE|BANK CHARGES|C2|DCGSSBOY|DCGSSGIRL|DOT|gift_0001_|PADS|POST|M')]

In [0]:
#Checking shape of data
data.shape

(396070, 8)

### Question 1: 

Select only the transactions that have occured during the period 09/01/11 and 11/30/11 and create a new subset of data. For answering Questions 2 to 5, use this subset of data.

In [0]:
#Subsetting dataframe making use of the InvoiceDate column
Data_Subset = data[(data['InvoiceDate'] >= '2011-09-01') & (data['InvoiceDate'] <= '2011-11-30')]

#resetting index of the dataframe
Data_Subset.reset_index(inplace=True, drop=True)

In [0]:
#Viewing the result
Data_Subset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,565080,20677,PINK POLKADOT BOWL,8,2011-09-01 08:25:00,1.25,13509.0,United Kingdom
1,565080,22128,PARTY CONES CANDY ASSORTED,24,2011-09-01 08:25:00,1.25,13509.0,United Kingdom
2,565082,22423,REGENCY CAKESTAND 3 TIER,2,2011-09-01 09:15:00,12.75,13305.0,United Kingdom
3,565082,15060B,FAIRY CAKE DESIGN UMBRELLA,8,2011-09-01 09:15:00,3.75,13305.0,United Kingdom
4,565082,23245,SET OF 3 REGENCY CAKE TINS,4,2011-09-01 09:15:00,4.95,13305.0,United Kingdom


In [0]:
#Checking if the oldest row of this dataset pertains to 09/01/11 or later
Data_Subset['InvoiceDate'].min()

Timestamp('2011-09-01 08:25:00')

In [0]:
#Checking if the latest row of this dataset pertains to 11/30/11 or older
Data_Subset['InvoiceDate'].max()

Timestamp('2011-11-29 18:14:00')

The dataset is now ready for answering Questions 2 to 5.

### Question 2:

Transform the above dataset in Question 1 into another dataset where each record represents an invoice. The record will have an InvoiceNo as column and StockCodes of the corresponding items bought in the invoice. The number of StockCodes in an invoice can be a variable.

In [0]:
#Transformed Dataset
basket = (Data_Subset
          .groupby(['InvoiceNo', 'StockCode'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [0]:
#Taking a look at the result
basket

StockCode,10080,10120,10124A,10124G,10125,10133,10135,11001,15030,15034,15036,15039,15044A,15044B,15044C,15044D,15056BL,15056N,15056P,15058A,15058B,15058C,15060B,16008,16011,16012,16014,16015,16016,16045,16046,16048,16054,16156L,16156S,16161G,16161P,16161U,16162L,16169E,...,90199B,90199C,90199D,90200A,90200B,90200D,90200E,90201A,90201B,90201C,90201D,90202A,90202B,90202C,90202D,90204,90205A,90205C,90206A,90206C,90208,90209A,90209B,90209C,90210B,90210D,90211A,90211B,90212C,90214A,90214C,90214D,90214E,90214G,90214H,90214J,90214K,90214L,90214N,90214S
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
565080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
565082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
565083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
565084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
565086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Question 3:

Prepare the data and generate the association rules from the above dataset in Question 2. Filter out all the rules by minimum support of 0.01 and lift of more than 1.0

In [0]:
#Importing necessary libraries
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and those that are 0 is set to 0. 

In [0]:
#This step will complete the one hot encoding of the data

#Making use of lambda function to house the if-else condition and
#finally map it to the basket sets
basket_sets = basket.applymap(lambda x: 1 if x >= 1 else 0)

Now that the data is structured properly, we can generate frequent item sets that have a support of at least 1% (0.01)

In [0]:
#Using min_support = 0.01
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [0]:
#Printing 10 randomly 
frequent_itemsets.sample(10, random_state=90)

,support,itemsets
1429,0.011156,"(20728, 22384, 20727)"
189,0.011964,(22121)
558,0.045918,(23293)
327,0.011318,(22645)
875,0.017462,"(21485, 22111)"
433,0.010348,(22995)
595,0.032660,(23350)
703,0.021180,(48138)
832,0.014551,"(23207, 20727)"
1456,0.011156,"(22866, 22865, 22111)"


The final step is to generate the rules with their corresponding support, confidence and lift:

In [0]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(23203),(20712),0.078254,0.025869,0.010509,0.134298,5.191439,0.008485,1.125249
1,(20712),(23203),0.025869,0.078254,0.010509,0.406250,5.191439,0.008485,1.552415
2,(85099B),(20712),0.088763,0.025869,0.012773,0.143898,5.562557,0.010477,1.137868
3,(20712),(85099B),0.025869,0.088763,0.012773,0.493750,5.562557,0.010477,1.799974
4,(20724),(20719),0.042037,0.023929,0.014228,0.338462,14.144491,0.013222,1.475456


### Question 4:

Find the top 10 rules from the above association rule set, sorted by confidence in descending order.

In [0]:
#Top 10 rules sorted by confidence in descending order
rules.sort_values('confidence', ascending=False)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1892,"(23170, 22697)",(23171),0.010833,0.016492,0.010186,0.940299,57.017120,0.010007,16.473767
1898,"(23171, 22699)",(23170),0.011318,0.018755,0.010348,0.914286,48.748768,0.010135,11.447858
1904,"(22725, 22727)",(22726),0.013096,0.052385,0.011964,0.913580,17.439796,0.011278,10.965262
1778,"(22577, 22579)",(22578),0.030719,0.059984,0.027971,0.910526,15.179529,0.026128,10.506063
1746,"(22386, 85099F)",(85099B),0.011318,0.088763,0.010186,0.900000,10.139344,0.009181,9.112369
905,(23172),(23171),0.012288,0.016492,0.010994,0.894737,54.254386,0.010792,9.343331
786,(22746),(22748),0.011318,0.017623,0.010024,0.885714,50.258191,0.009825,8.595796
1537,"(22578, 22086)",(22577),0.012611,0.059337,0.011156,0.884615,14.908300,0.010408,8.152412
902,(23172),(23170),0.012288,0.018755,0.010833,0.881579,47.004877,0.010602,8.286068
1562,"(22910, 23319)",(22086),0.014390,0.108973,0.012611,0.876404,8.042377,0.011043,7.209216


### Question 5:

Repeat the complete exercise from Question 1 to Question 4 for all the invoices of country France only for the period between 01/12/2010 and 09/12/2011. Find the top 10 rules from the above association rule set, sorted by confidence in descending order. 

In [0]:
#Since we do not need the time frame that was taken in Question 1, we can make 
#use of the code template from Question 2 onwards.

#Transformed Dataset: basket_France
basket_France = (data[data['Country'] =="France"]
          .groupby(['InvoiceNo', 'StockCode'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [0]:
#Taking a look at the result
basket_France

StockCode,10002,10120,10125,10135,11001,15036,15039,15044C,15056BL,15056N,15056P,15058A,15058B,15058C,16012,16048,16156L,16156S,16161P,16161U,16169E,16218,16219,16225,16236,16237,16238,16258A,17011F,17012A,17012B,17174,20615,20617,20658,20665,20668,20674,20675,20676,...,85049H,85053,85059,85066,85088,85095,85099B,85099C,85099F,85114C,85116,85123A,85130C,85135C,85150,85159B,85169B,85169D,85170D,85172,85173,85175,85183B,85194S,85199L,85199S,85206A,85227,85230B,85230E,85230G,85232D,90030B,90030C,90031,90099,90184B,90184C,90201B,90201C
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536370,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and those that are 0 is set to 0. 

In [0]:
#This step will complete the one hot encoding of the data
basket_sets_France = basket_France.applymap(lambda y: 1 if y >= 1 else 0)

Now that the data is structured properly, we can generate frequent item sets that have a support of at least 1% (0.01)

In [0]:
#Using min_support = 0.01
frequent_itemsets_France = apriori(basket_sets_France, min_support=0.01, use_colnames=True)

In [0]:
#Printing 10 randomly 
frequent_itemsets_France.sample(10, random_state=90)

,support,itemsets
13296,0.010554,"(22556, 22303, 22659, 20750)"
14321,0.010554,"(22728, 21086, 23256, 22492)"
8974,0.010554,"(23190, 22326, 22554)"
7069,0.010554,"(22029, 21086, 22554)"
3299,0.013193,"(22396, 22398)"
29744,0.010554,"(22727, 22326, 21086, 22726, 21080, 21094, 225..."
14911,0.010554,"(22728, 23199, 21558, 23254)"
19759,0.010554,"(22326, 23254, 22352, 20750, 23291)"
29506,0.010554,"(22727, 22326, 22726, 21558, 20750, 22728, 232..."
22298,0.010554,"(22556, 22326, 22551, 23190, 23241)"


The final step is to generate the rules with their corresponding support, confidence and lift:

In [0]:
rules_France = association_rules(frequent_itemsets_France, metric="lift", min_threshold=1)
rules_France.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(21791),(10002),0.029024,0.021108,0.010554,0.363636,17.227273,0.009941,1.538259
1,(10002),(21791),0.021108,0.029024,0.010554,0.500000,17.227273,0.009941,1.941953
2,(21915),(10002),0.071240,0.021108,0.010554,0.148148,7.018519,0.009050,1.149134
3,(10002),(21915),0.021108,0.071240,0.010554,0.500000,7.018519,0.009050,1.857520
4,(22551),(10002),0.139842,0.021108,0.010554,0.075472,3.575472,0.007602,1.058801


In [0]:
#Top 10 rules sorted by confidence in descending order
rules_France.sort_values('confidence', ascending=False)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
576162,"(23291, 22352, 22727, 22659)","(22728, 20750, 23254)",0.010554,0.013193,0.010554,1.0,75.800000,0.010415,inf
556492,"(23254, 22352, 22659, 20750, 22728)","(22726, 22326)",0.010554,0.050132,0.010554,1.0,19.947368,0.010025,inf
556483,"(22326, 22726, 22352, 22659, 20750)","(22728, 23254)",0.010554,0.021108,0.010554,1.0,47.375000,0.010331,inf
556484,"(22326, 22726, 22352, 22659, 22728)","(20750, 23254)",0.010554,0.021108,0.010554,1.0,47.375000,0.010331,inf
556486,"(22326, 22726, 22659, 20750, 22728)","(22352, 23254)",0.010554,0.015831,0.010554,1.0,63.166667,0.010387,inf
556487,"(22326, 22352, 22659, 20750, 22728)","(22726, 23254)",0.010554,0.018470,0.010554,1.0,54.142857,0.010359,inf
556488,"(23254, 22726, 22352, 22659, 20750)","(22728, 22326)",0.010554,0.050132,0.010554,1.0,19.947368,0.010025,inf
556489,"(23254, 22726, 22352, 22659, 22728)","(22326, 20750)",0.010554,0.052770,0.010554,1.0,18.950000,0.009997,inf
556490,"(23254, 22726, 22352, 20750, 22728)","(22659, 22326)",0.010554,0.034301,0.010554,1.0,29.153846,0.010192,inf
556491,"(23254, 22726, 22659, 20750, 22728)","(22352, 22326)",0.010554,0.044855,0.010554,1.0,22.294118,0.010081,inf


### Answer Questions 6 to 8 using the datasets *ratings.csv* and *movies.csv*, which are already used in the chapter. 

Link: https://grouplens.org/datasets/movielens/

#### Data Exploration

##### ratings.csv:

In [0]:
#import necessary libraries
import pandas as pd
import numpy as np

#loading dataset
rating_df = pd.read_csv('/content/gdrive/My Drive/My solution for Chapter 9 (Machine Learning Using Python)/ratings.csv')

In [0]:
#print the first 5 records
rating_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [0]:
rating_df.shape

(25000095, 4)

In [0]:
#Checking for missing values (if any)
rating_df.isnull().sum(axis=0)

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

##### movies.csv:

In [0]:
#loading the ratings file
movies_df = pd.read_csv('/content/gdrive/My Drive/My solution for Chapter 9 (Machine Learning Using Python)/movies.csv')

In [0]:
#print the first 5 records
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
#Checking the shape of the dataset
movies_df.shape

(62423, 3)

In [0]:
#Checking for missing values (if any)
movies_df.isnull().sum(axis=0)

movieId    0
title      0
genres     0
dtype: int64

### Question 6:

Filter out the ratings records for the movies that belong to only "Action" genre. Calculate item similarity indexes between the movies based on movies the users have bought in "Action" genre and how they have rated them. Use Pearson correlation coefficient to find similarities. Then recommend top 5 similar movies to the following movies:

**a.** Heat

**b.** Eraser

In [0]:
#Subsetting movies to the ones that have "Action" mentioned in the genre
movies_action = movies_df[movies_df['genres'].str.contains("Action")]

#resetting index of the dataframe
movies_action.reset_index(inplace=True, drop=True)

In [0]:
#printing shape of the dataset
movies_action.shape

(7348, 3)

In [0]:
#Subsetting ratings dataset to only "Action" genre using an inner join with the subsetted movies dataset
#This also gives us a common dataset to work on
ratings_movies_action = pd.merge(rating_df, movies_action, on='movieId', how='inner')

#resetting index of the dataframe
ratings_movies_action.reset_index(inplace=True, drop=True)

In [0]:
#printing top 5 records of the resulting dataset
ratings_movies_action.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2692,5.0,1147869100,Run Lola Run (Lola rennt) (1998),Action|Crime
1,12,2692,5.0,940766481,Run Lola Run (Lola rennt) (1998),Action|Crime
2,23,2692,4.0,945753888,Run Lola Run (Lola rennt) (1998),Action|Crime
3,43,2692,3.5,1170493783,Run Lola Run (Lola rennt) (1998),Action|Crime
4,50,2692,4.5,1402504842,Run Lola Run (Lola rennt) (1998),Action|Crime


Calculating item similarity indexes between the movies based on movies the users have bought in "Action" genre and how they have rated them.

In [0]:
#import libraries
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import correlation

In [0]:
#Create a matrix where rows represent movies, columns represent users and cell values represent ratings
rating_mat = ratings_movies_action.pivot(index='movieId',
                                        columns='userId',
                                        values = 'rating').reset_index(drop=True)

In [0]:
#Fill all NaNs with 0
rating_mat.fillna(0, inplace = True)

In [0]:
#Finding correlation between movies
movie_sim = 1 - pairwise_distances(rating_mat.values, metric = "correlation")

In [0]:
#converting to a dataframe
movie_sim_df = pd.DataFrame(movie_sim)

In [0]:
#print the first 5 movies
movie_sim_df.iloc[0:5,0:5]

,0,1,2,3,4
0,1.000000,0.192946,0.252661,0.113946,0.178166
1,0.192946,1.000000,0.122988,0.161707,0.250714
2,0.252661,0.122988,1.000000,0.178747,0.190606
3,0.113946,0.161707,0.178747,1.000000,0.235620
4,0.178166,0.250714,0.190606,0.235620,1.000000


In [0]:
#creating a copy of the dataset ratings_movies_action
Temp = ratings_movies_action

#finding the substring "Heat" in the list of titles and inserting the corresponding index values as a separate column
#Index=-1 represents Not found else the corresponding index value is returned
Temp['index_Heat'] = Temp['title'].str.find('Heat')

#finding the substring "Eraser" in the list of titles and inserting the corresponding index values as a separate column
#Index=-1 represents Not found else the corresponding index value is returned
Temp['index_Eraser'] = Temp['title'].str.find('Eraser')

#defining a function to return the list of movie tites based on substring used
#Here we are interested in perfect match (where we need the word found at index 0)
def Actual_movie_name(index_substring):
    return Temp[Temp[index_substring] == 0]

### Question 6(a)

Heat

In [0]:
#Calling the function to retrieve records based on positive match for "Heat" substring
Heat = Actual_movie_name("index_Heat")

In [0]:
#Printing only the unique values from the big list
Heat['title'].unique()

array(['Heat (1995)', 'Heat, The (2013)', 'Heat (1986)',
       'Heat Lightning (1934)', 'Heathens and Thieves (2012)'],
      dtype=object)

### From the output above we need to query for two particular strings:
### "Heat (1995)" and "Heat (1986)"

In [0]:
#extracting row information of the movie title: Heat (1995)
Heat[Heat['title'] == 'Heat (1995)']

,userId,movieId,rating,timestamp,title,genres,index_Heat,index_Eraser
4563730,8,6,3.0,890489713,Heat (1995),Action|Crime|Thriller,0,-1
4563731,23,6,4.0,942967473,Heat (1995),Action|Crime|Thriller,0,-1
4563732,41,6,4.0,944572130,Heat (1995),Action|Crime|Thriller,0,-1
4563733,48,6,5.0,849969301,Heat (1995),Action|Crime|Thriller,0,-1
4563734,72,6,4.0,982621651,Heat (1995),Action|Crime|Thriller,0,-1
...,...,...,...,...,...,...,...,...
4588313,162516,6,4.5,1175652998,Heat (1995),Action|Crime|Thriller,0,-1
4588314,162518,6,3.0,869634332,Heat (1995),Action|Crime|Thriller,0,-1
4588315,162519,6,2.0,1000946676,Heat (1995),Action|Crime|Thriller,0,-1
4588316,162529,6,2.0,888181575,Heat (1995),Action|Crime|Thriller,0,-1


In [0]:
#extracting row information of the movie title: Heat (1986)
Heat[Heat['title'] == 'Heat (1986)']

,userId,movieId,rating,timestamp,title,genres,index_Heat,index_Eraser
7424909,5865,131274,1.0,1500394868,Heat (1986),Action|Drama|Thriller,0,-1
7424910,29803,131274,3.0,1545119122,Heat (1986),Action|Drama|Thriller,0,-1
7424911,32349,131274,3.0,1536955740,Heat (1986),Action|Drama|Thriller,0,-1
7424912,33844,131274,2.0,1436135324,Heat (1986),Action|Drama|Thriller,0,-1
7424913,78849,131274,2.5,1427807031,Heat (1986),Action|Drama|Thriller,0,-1
7424914,80914,131274,3.5,1546081699,Heat (1986),Action|Drama|Thriller,0,-1
7424915,115102,131274,2.5,1466987503,Heat (1986),Action|Drama|Thriller,0,-1


Now that we have the movieId as well for these two movies with the same name: Heat. We can make use of it to retrieve rows (similar movies) of our dataset based on similarity index.



1.   Movie: Heat (1995) -> MovieID: 6
2.   Movie: Heat (1986) -> MovieID: 131274



In [0]:
#function to find most similar movies

def get_similar_movies(movieid, topN = 5):
    #Get the index of the movie record in the movies_df
    movieidx = movies_action[movies_action.movieId == movieid].index[0]
    movies_action['similarity'] = movie_sim_df.iloc[movieidx]
    top_n = movies_action.sort_values(["similarity"], ascending=False)[0:topN]
    return top_n

In [0]:
#Getting similar movies for Movie: Heat (1995) -> MovieID: 6
get_similar_movies(6)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,movieId,title,genres,similarity
0,6,Heat (1995),Action|Crime|Thriller,1.000000
112,733,"Rock, The (1996)",Action|Adventure|Thriller,0.381449
16,95,Broken Arrow (1996),Action|Adventure|Thriller,0.355552
104,648,Mission: Impossible (1996),Action|Adventure|Mystery|Thriller,0.343545
76,474,In the Line of Fire (1993),Action|Thriller,0.315993


In [0]:
#Getting similar movies for Movie: Heat (1986) -> MovieID: 131274
get_similar_movies(131274)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,movieId,title,genres,similarity
3609,131274,Heat (1986),Action|Drama|Thriller,1.000000
4195,142162,The Payoff (1942),Action|Crime|Mystery|Thriller,0.246823
4166,141538,Assassination (2015),Action|Drama|Thriller,0.241009
4160,141402,Star Trek: Renegades (2015),Action|Adventure|Sci-Fi,0.240507
2946,117504,Crackers (1984),Action|Comedy|Crime|Thriller,0.233528


### Question 6(b)

Eraser

In [0]:
#Calling the function to retrieve the positive match for "Eraser" substring
Eraser = Actual_movie_name("index_Eraser")

In [0]:
#Printing only the unique values from the big list
Eraser['title'].unique()

array(['Eraser (1996)'], dtype=object)

### From the output above we need to query for just "Eraser (1996)"

In [0]:
#extracting row information of the movie title: Eraser (1996)
Eraser[Eraser['title'] == 'Eraser (1996)']

,userId,movieId,rating,timestamp,title,genres,index_Heat,index_Eraser
4443825,5,786,2.0,858625935,Eraser (1996),Action|Drama|Thriller,-1,0
4443826,8,786,5.0,890490020,Eraser (1996),Action|Drama|Thriller,-1,0
4443827,41,786,3.0,944573951,Eraser (1996),Action|Drama|Thriller,-1,0
4443828,48,786,4.0,849969361,Eraser (1996),Action|Drama|Thriller,-1,0
4443829,105,786,4.0,850678951,Eraser (1996),Action|Drama|Thriller,-1,0
...,...,...,...,...,...,...,...,...
4459232,162516,786,3.0,1074286964,Eraser (1996),Action|Drama|Thriller,-1,0
4459233,162518,786,4.0,869634333,Eraser (1996),Action|Drama|Thriller,-1,0
4459234,162519,786,3.0,1000947326,Eraser (1996),Action|Drama|Thriller,-1,0
4459235,162521,786,2.0,1207989520,Eraser (1996),Action|Drama|Thriller,-1,0


Now that we have the movieId for this movie: Eraser (1996). We can make use of it to retrieve rows (similar movies) of our dataset based on similarity index.



1.   Movie: Eraser (1996) -> MovieID: 786



In [0]:
#Getting similar movies for Movie: Eraser (1996) -> MovieID: 786
get_similar_movies(786)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,movieId,title,genres,similarity
118,786,Eraser (1996),Action|Drama|Thriller,1.000000
16,95,Broken Arrow (1996),Action|Adventure|Thriller,0.473542
81,494,Executive Decision (1996),Action|Adventure|Thriller,0.471779
112,733,"Rock, The (1996)",Action|Adventure|Thriller,0.454496
113,736,Twister (1996),Action|Adventure|Romance|Thriller,0.433409


### Question 7:

Filter out the ratings records for the movies that belong to either "Animation" or "Children" genre. And Calculate item similarity indexes between the movies based on movies the users have bought in either "Animation" or "Children" genre and how they have rated them. Use cosine similarity index to find similarities. Then recommend top 5 similar movies to the following movies:

**a.** Lion King

**b.** The Incredibles

In [0]:
#Subsetting movies to the ones that have "Animation" or "Children" mentioned in the genre
movies_animation_children = movies_df[movies_df['genres'].str.contains("Animation|Children")]

#resetting index of the dataframe
movies_animation_children.reset_index(inplace=True, drop=True)

In [0]:
#printing shape of the dataset
movies_animation_children.shape

(4849, 3)

In [0]:
#Subsetting ratings dataset to only "Animation" or "Children" genre using an inner join with the subsetted movies dataset
#This also gives us a common dataset to work on
ratings_movies_animation_children = pd.merge(rating_df, movies_animation_children, on='movieId', how='inner')

#resetting index of the dataframe
ratings_movies_animation_children.reset_index(inplace=True, drop=True)

In [0]:
#printing top 5 records of the resulting dataset
ratings_movies_animation_children.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2161,3.5,1147868609,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy
1,26,2161,3.5,1280513623,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy
2,60,2161,2.0,939595135,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy
3,67,2161,5.0,975662347,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy
4,72,2161,4.0,980644882,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy


Calculating item similarity indexes between the movies based on movies the users have bought in "Animation" or "Children" genre and how they have rated them.

In [0]:
#import libraries
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import correlation

In [0]:
#Create a matrix where rows represent movies, columns represent users and cell values represent ratings
rating_mat_two = ratings_movies_animation_children.pivot(index='movieId',
                                        columns='userId',
                                        values = 'rating').reset_index(drop=True)

In [0]:
#Fill all NaNs with 0
rating_mat_two.fillna(0, inplace = True)

In [0]:
#Finding correlation between movies
movie_sim_two = 1 - pairwise_distances(rating_mat_two.values, metric = "cosine")

In [0]:
#converting to a dataframe
movie_sim_df_two = pd.DataFrame(movie_sim_two)

In [0]:
#print the first 5 movies
movie_sim_df_two.iloc[0:5,0:5]

,0,1,2,3,4
0,1.000000,0.399026,0.097002,0.129309,0.102428
1,0.399026,1.000000,0.172029,0.163789,0.147212
2,0.097002,0.172029,1.000000,0.176906,0.209600
3,0.129309,0.163789,0.176906,1.000000,0.083432
4,0.102428,0.147212,0.209600,0.083432,1.000000


In [0]:
#creating a copy of the dataset ratings_movies_animation_children
Temp_two = ratings_movies_animation_children

#finding the substring "Lion King" in the list of titles and inserting the corresponding index values as a separate column
#Index=-1 represents Not found else the corresponding index value is returned
Temp_two['index_Lion_King'] = Temp_two['title'].str.find('Lion King')

#finding the substring "Incredibles" in the list of titles and inserting the corresponding index values as a separate column
#Index=-1 represents Not found else the corresponding index value is returned
Temp_two['index_The_Incredibles'] = Temp_two['title'].str.find('Incredibles')

#defining a function to return the list of movie tites based on substring used
#Here we are interested in perfect match (where we need the word found at index 0)
def Actual_movie_name_two(index_substring):
    return Temp_two[Temp_two[index_substring] == 0]

### Question 7(a)

Lion King

In [0]:
#Calling the function to retrieve records based on positive match for "Lion King" substring
Lion_King = Actual_movie_name_two('index_Lion_King')

In [0]:
#Printing only the unique values from the big list
Lion_King['title'].unique()

array(['Lion King, The (1994)'], dtype=object)

### From the output above we need to query for just 'Lion King, The (1994)'

In [0]:
#extracting row information of the movie title: 'Lion King, The (1994)'
Lion_King[Lion_King['title'] == 'Lion King, The (1994)']

,userId,movieId,rating,timestamp,title,genres,index_Lion_King,index_The_Incredibles
122774,2,364,4.5,1141417077,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1
122775,5,364,4.0,831900357,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1
122776,8,364,4.0,890489482,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1
122777,12,364,3.0,1119354644,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1
122778,13,364,4.0,1265224015,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1
...,...,...,...,...,...,...,...,...
165514,162524,364,3.5,1072919314,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1
165515,162529,364,3.0,888184021,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1
165516,162530,364,3.0,989809966,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1
165517,162533,364,4.5,1281406011,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0,-1


Now that we have the movieId for this movie: Lion King, The (1994). We can make use of it to retrieve rows (similar movies) of our dataset based on similarity index.



1.   Movie: 'Lion King, The (1994)' -> MovieID: 364



In [0]:
#function to find most similar movies (Note: This function is not the same as before)

def get_similar_movies_animation_children(movieid_two, topN_two = 5):
    #Get the index of the movie record in the movies_df
    movieidx_two = movies_animation_children[movies_animation_children.movieId == movieid_two].index[0]
    movies_animation_children['similarity'] = movie_sim_df_two.iloc[movieidx_two]
    top_n_two = movies_animation_children.sort_values(["similarity"], ascending=False)[0:topN_two]
    return top_n_two

In [0]:
#Getting similar movies for Movie: 'Lion King, The (1994)' -> MovieID: 364
get_similar_movies_animation_children(364)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,movieId,title,genres,similarity
32,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,1.000000
47,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,0.687329
49,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,0.667784
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.530681
46,586,Home Alone (1990),Children|Comedy,0.522619


### Question 7(b)

The Incredibles

In [0]:
#Calling the function to retrieve the positive match for "The Incredibles" substring
The_Incredibles = Actual_movie_name('index_The_Incredibles')

In [0]:
#Printing only the unique values from the big list
The_Incredibles['title'].unique()

array(['Incredibles, The (2004)', 'Incredibles 2 (2018)'], dtype=object)

### From the output above we need to query for just 'Incredibles, The (2004)'

In [0]:
#extracting row information of the movie title: 'Incredibles, The (2004)'
The_Incredibles[The_Incredibles['title'] == 'Incredibles, The (2004)']

,userId,movieId,rating,timestamp,title,genres,index_Lion_King,index_The_Incredibles
493413,2,8961,4.5,1141415946,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0
493414,3,8961,4.0,1439472667,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0
493415,4,8961,5.0,1573938092,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0
493416,12,8961,3.0,1183550529,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0
493417,13,8961,4.5,1237970263,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0
...,...,...,...,...,...,...,...,...
523970,162533,8961,4.0,1280920512,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0
523971,162534,8961,4.5,1526714124,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0
523972,162538,8961,2.5,1438781529,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0
523973,162540,8961,4.0,1249028593,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,-1,0


Now that we have the movieId for this movie: Incredibles, The (2004). We can make use of it to retrieve rows (similar movies) of our dataset based on similarity index.



1.   Movie: Incredibles, The (2004) -> MovieID: 8961



In [0]:
#Getting similar movies for Movie: 'Incredibles, The (2004)' -> MovieID: 8961
get_similar_movies_animation_children(8961)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,movieId,title,genres,similarity
490,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,1.000000
399,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,0.670137
327,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,0.639807
308,4306,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Ro...,0.593805
459,8360,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...,0.550839


### Question 8:

Use *surprise* library and grid search mechanism to find the best model to make recommendations for movies in "Action" genre only. Use the following possible parameters to search for best model:

**a.** Number of neighbors [5, 10, 20].

**b.** Similarity indexes ['Cosine', 'Pearson', 'Euclidean'].

**c.** User-based or item-based similarity.


In [0]:
#Subsetting movies to the ones that have "Action" only mentioned in the genre
movies_action_only = movies_df[movies_df['genres'] == "Action"]

#resetting index of the dataframe
movies_action_only.reset_index(inplace=True, drop=True)

In [0]:
#Subsetting ratings dataset to only "Action" genre using an inner join with the subsetted movies dataset
#This also gives us a common dataset to work on
ratings_movies_action_only = pd.merge(rating_df, movies_action_only, on='movieId', how='inner')

#resetting index of the dataframe
ratings_movies_action_only.reset_index(inplace=True, drop=True)

In [0]:
#Installing library: surprise
 !pip install surprise

     |████████████████████████████████| 6.5MB 3.2MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678205 sha256=142339d5931c856c1e7a345b3d154d68d1fddad1d7fe6c77bb81e2aeace27aa8
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
#importing required modules or classes from library: surprise
from surprise import Dataset, Reader, KNNBasic, accuracy

In [0]:
#Reader class to provide the range of rating scales that is being used 
reader = Reader(rating_scale=(1,5))

#surprise.Dataset to load the dataset
#load_from_df to convert DataFrame to a Dataset
data = Dataset.load_from_df(ratings_movies_action_only[['userId',
                                       'movieId',
                                       'rating']], reader=reader)

## Finding the Best Model

**a.** Number of neighbors [5, 10, 20].

**b.** Similarity indexes ['Cosine', 'Pearson', 'Euclidean'].

**c.** User-based or item-based similarity.

In [0]:
#Importing GridSearchCV from surprise
from surprise.model_selection.search import GridSearchCV

In [0]:
#Setting the param_grid per mentioned options
param_grid = {'k':[5, 10, 20],
              'sim_options': {'name': ['cosine', 'pearson', 'msd'],
                              'user_based': [True, False]
                              }
              }

In [0]:
#Setting up GridSearchCV to return the best model and its parameters
grid_cv = GridSearchCV(KNNBasic,
                       param_grid,
                       measures=['rmse'],
                       cv=5,
                       refit=True)

In [22]:
#Fitting data
grid_cv.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [23]:
#Best RMSE score
print(grid_cv.best_score['rmse'])

#Combination of parameters that gave the best RMSE score
print(grid_cv.best_params['rmse'])

0.989181692282617
{'k': 20, 'sim_options': {'name': 'msd', 'user_based': True}}


The best model is user-based collaborative filtering with msd similarity and 20 similar users. Details of the grid search are captured in the variable cv_results. We can convert it to a DataFrame and print a few columns like param_sim_options and mean_test_rmse.

In [24]:
results_df = pd.DataFrame.from_dict(grid_cv.cv_results)
results_df[['param_k', 'param_sim_options', 'mean_test_rmse', 'rank_test_rmse']]

,param_k,param_sim_options,mean_test_rmse,rank_test_rmse
0,5,"{'name': 'cosine', 'user_based': True}",1.103348,18
1,5,"{'name': 'cosine', 'user_based': False}",1.044551,12
2,5,"{'name': 'pearson', 'user_based': True}",1.095558,17
3,5,"{'name': 'pearson', 'user_based': False}",1.042494,11
4,5,"{'name': 'msd', 'user_based': True}",1.013036,3
5,5,"{'name': 'msd', 'user_based': False}",1.031730,4
6,10,"{'name': 'cosine', 'user_based': True}",1.074017,14
7,10,"{'name': 'cosine', 'user_based': False}",1.041670,10
8,10,"{'name': 'pearson', 'user_based': True}",1.086563,16
9,10,"{'name': 'pearson', 'user_based': False}",1.040698,7
